In [109]:
import rospy
import numpy as np
import pinocchio as pin
import time
from sensor_msgs.msg import JointState
from std_msgs.msg import Float64MultiArray, Int32MultiArray
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from std_msgs.msg import Duration

In [209]:
class KinovaROS:
    SET_CONFIG_KP = 0
    SET_CONFIG_KI = 1
    SET_CONFIG_GAMMA = 2
    SET_CONFIG_POS_CTRL_MODE = 3
    NORMAL_PID = 0
    GRAVITY_COMPENSATION_ONLY = 1

    def __init__(self, node, vel_limit):
        self.node = node
        self.vel_limit = vel_limit
        self.joint_state = {}
        self.robot_state = {}
        self.joint_names = [f"joint_{i+1}" for i in range(7)]
        
        def joint_states_cb(msg):
            self.joint_state["q"] = np.array(msg.position)
            self.joint_state["dq"] = np.array(msg.velocity)
        def robot_states_cb(msg):
            self.robot_state["pos_ctrl_mode"] = msg.data[0]
            self.robot_state["is_traj_following"] = msg.data[1]
            self.robot_state["kp"] = msg.data[2:9]
            self.robot_state["ki"] = msg.data[9:16]
            self.robot_state["gamma"] = msg.data[16:23]

        self.joint_states_sub = rospy.Subscriber('/joint_state', JointState, joint_states_cb)
        self.robot_states_sub = rospy.Subscriber('/robot_state', Float64MultiArray, robot_states_cb)
        self.robot_config_pub = rospy.Publisher('/robot_config', Float64MultiArray, queue_size=1)
        self.trajectory_pub = rospy.Publisher('/joint_traj', JointTrajectory, queue_size=1)
        # self.joint_desired_pub = rospy.Publisher('/joint_state', JointState, queue_size=1)
        # self.pos_ctrl_mode_pub = rospy.Publisher('/pos_ctrl_mode', Int32MultiArray, queue_size=1)
    
    def set_pos_ctrl_mode(self, mode):
        msg = Float64MultiArray()
        if mode == "gravity":
            msg.data = [self.SET_CONFIG_POS_CTRL_MODE, self.GRAVITY_COMPENSATION_ONLY]
        elif mode == "pid":
            msg.data = [self.SET_CONFIG_POS_CTRL_MODE, self.NORMAL_PID]
        self.robot_config_pub.publish(msg)
    
    def set_ctrl_gain(self, gaintype, gain):
        assert len(gain) == 7
        if gaintype == "kp":
            config_enum = self.SET_CONFIG_KP
        elif gaintype == "ki":
            config_enum = self.SET_CONFIG_KI
        elif gaintype == "gamma":
            config_enum = self.SET_CONFIG_GAMMA
        
        self.set_pos_ctrl_mode("gravity")
        time.sleep(0.01)
        msg = Float64MultiArray()
        msg.data = [config_enum, *gain]
        self.robot_config_pub.publish(msg)
        time.sleep(0.01)
        self.set_pos_ctrl_mode("pid")
        
    
    def set_waypoints(self, msg:JointTrajectory):
        self.trajectory_pub.publish(msg)
    
    def generate_traj_msg(self, qs, last_timestep=None):
        if last_timestep is None:
            last_timestep = self.get_default_last_timestep(qs)
            print("last_timestep: ", last_timestep)
        num_step = len(qs)
        timesteps = np.linspace(0, last_timestep,num_step)

        waypoints = []
        w_init = JointTrajectoryPoint()
        w_init.positions = list(qs[0])
        w_init.velocities = [0.]*7
        w_init.time_from_start = w_init.time_from_start.from_sec(0.)

        w_goal = JointTrajectoryPoint()
        w_goal.positions = list(qs[-1])
        w_goal.velocities = [0.]*7
        w_goal.time_from_start = w_goal.time_from_start.from_sec(timesteps[-1])

        waypoints = [w_init]
        for i in range(1, len(timesteps)-1):
            w = JointTrajectoryPoint()
            w.positions = list(qs[i])
            w.velocities = list((qs[i+1] - qs[i-1])/(timesteps[i+1] - timesteps[i-1]))
            w.time_from_start = w.time_from_start.from_sec(timesteps[i])
            waypoints.append(w)
        waypoints.append(w_goal)

        traj = JointTrajectory()
        traj.points = waypoints
        traj.joint_names = self.joint_names
        return traj
    
    def get_default_last_timestep(self, qs, vel_const=0.3):
        distance = 0
        for i in range(len(qs)-1):
            distance += np.linalg.norm(qs[i] - qs[i+1])
        return distance/vel_const
        
    def goto(self, q_d, last_timestep=4):
        q_curr = self.joint_state['q']
        msg = self.generate_traj_msg([q_curr, q_d], last_timestep)
        self.set_waypoints(msg)

In [3]:
node = rospy.init_node('node1', anonymous=True)

In [211]:
KINOVA_URDF_PATH = "/home/pandacom/ws/kinova_joint_pid/robot_model/gen3_7dof/urdf/GEN3_URDF_V12_with_Hand_e_rev.urdf"
model    = pin.buildModelFromUrdf(KINOVA_URDF_PATH)
data     = model.createData()
robot = KinovaROS(node, model.velocityLimit)

In [214]:
q1 = robot.joint_state["q"]

In [216]:
q2 = robot.joint_state["q"]

In [215]:
robot.set_pos_ctrl_mode('gravity')

In [42]:
kp_curr = robot.robot_state["kp"]

In [ ]:
robot.get_default_last_timestep()

In [217]:
qs = [q2, q1, q2]
msg = robot.generate_traj_msg(qs)

last_timestep:  4.25976839674024


In [218]:
robot.set_waypoints(msg)

In [176]:
robot.goto(q1, last_timestep=1.5)

2.294883764421756


In [205]:
traj = [q1,q2,q1]
msg = robot.generate_traj_msg([q1,q2,q1], last_timestep=2.3)
robot.set_waypoints(msg)

In [168]:
# test gain.
robot.set_ctrl_gain("kp", [60, 60, 60, 60, 50, 50, 50])
robot.set_ctrl_gain("ki", [900,900,900,900,625,625,625])
robot.set_ctrl_gain("gamma", [0.1,0.1,0.1,0.1,0.2,0.2,0.2,])

In [ ]:
robot.set_waypoints(traj)